# Hit Path 정제

20.03.09  
User의 App 이동경로를 정제하여 간략화하는 작업.  
User별 이동경로 파악.

In [1]:
import pandas as pd
import re
import warnings
warnings.filterwarnings(action = 'ignore')

데이터 불러오기

In [2]:
rv = pd.read_excel('Hit Path Data.xlsx')

값이 없는 행 제거하기

In [3]:
rv = rv[rv['hit_path'].notnull()][['DATE', 'client_id', 'hit_path']].reset_index()

In [4]:
# 글자 제거
def hptc(b, a):
    cc = re.compile(' ')
    rv['hit_path'] = rv['hit_path'].apply(lambda x : cc.sub('', x))    
    
    cc = re.compile('>')
    rv['hit_path'] = rv['hit_path'].apply(lambda x : cc.sub(' > ', x))    
    
    cc = re.compile('  ')
    rv['hit_path'] = rv['hit_path'].apply(lambda x : cc.sub(' ', x))  
    
    cc = re.compile(b)
    rv['hit_path'] = rv['hit_path'].apply(lambda x : cc.sub(a, x))    

In [5]:
# 웹/앱 구분
hptc('\[PAGE\]/', '[PAGE](웹)')

# 숫자 다 없애기 
hptc('\d+', '')

# /-/ 없애기
hptc('/-/', '')

# 스크롤 깊이 다 없애기
hptc('스크롤깊이///', '')

# <>안에 든거 없애기
hptc('<\S+ > ', '')

# contents_city / _bucket / _stay / _activity
hptc('/contents/', '_')

# main_bucket을 main으로 합치기
hptc('\[PAGE\]\(웹\)main > \[PAGE\]\(웹\)main_bucket', '[PAGE](웹)main')

# search_city / _stay / _bucket / _activity
hptc('search/city', 'search_city/')
hptc('search/stay', 'search_stay/')
hptc('search/bucket', 'search_bucket/')
hptc('search/activity', 'search_activity/')

# 시작이 랜딩인 것 정리하기
hptc('\?shortlink\S*', 'landing')

# 나머지 랜딩페이지 모두 정리하기
hptc('/landing\S*', '_landing')
hptc('\?af\S*', 'landing')

# tutorial
hptc('\[PAGE\]tutorial.*코치마크', '[PAGE]tutorial')

# newresult 줄이기
hptc('/newresult\S*', '_newresult')
# newresult 없애기
#hptc('/newresult\S*', '')

# n_media 줄이기
hptc('\?n_media\S*', 'n_media')

# // → /로 줄이기
hptc('//', '/')

# [EVENT] 모두 삭제하기
hptc(' > \[EVENT\]\S*', '')

# 첫번째를 제외한 나머지 제거하기
hptc('/\S*', '')

In [6]:
# 양쪽 공백 제거
rv['hit_path'] = rv['hit_path'].str.strip()

# 꺽쇠를 기준으로 분리하기
rv['split'] = rv['hit_path'].apply(lambda x: x.split(' > '))

In [7]:
# 같은 행동이 반복된다면 통합하기
result = []
for j in range(0, len(rv['split'])):
    temp = [rv['split'][j][0]]
    for i in range(0, len(rv['split'][j])-1):
        if rv['split'][j][i] != rv['split'][j][i+1]:
            temp.append(rv['split'][j][i+1])
    result.append(temp)

In [8]:
result = pd.DataFrame(result)
result['index'] = rv['index']
rv = pd.merge(rv, result, on = 'index', how = 'left')

---

User별 이동경로 나타내기

In [9]:
rv = rv.fillna('')
result = pd.pivot_table(rv, values = None, index = (rv[0], rv[1], rv[2], rv[3], rv[4], rv[5]))

# 데이터 예시
result.head(5)

DATE  \
0                   1                     2                    3                         4                     5                             
[EVENT]stay_landing [PAGE](웹)main         [PAGE](웹)bucket      [PAGE](웹)search_stay      [PAGE](웹)stay_landing [PAGE](웹)event   20191217.0   
                    [PAGE](웹)main_bucket  [PAGE](웹)search_stay [PAGE](웹)stay_landing                                            20191217.0   
                    [PAGE](웹)stay_landing                                                                                       20200133.5   
                                          [PAGE](웹)search      [PAGE](웹)search_newresult [PAGE](웹)stay_landing [PAGE](웹)search  20200101.0   
[EVENT]검색결과                                                                                                                     20200151.0   

                                                                                                                                 index  
0                   1                     2                    3                         4                     5                        
[EVENT]stay_landing [PAGE](웹)main         [PAGE](웹)bucket      [PAGE](웹)search_stay      [PAGE](웹)stay_landing [PAGE](웹)event    132.0  
                    [PAGE](웹)main_bucket  [PAGE](웹)search_stay [PAGE](웹)stay_landing                                             127.0  
                    [PAGE](웹)stay_landing                                                                                       1784.5  
                                          [PAGE](웹)search      [PAGE](웹)search_newresult [PAGE](웹)stay_landing [PAGE](웹)search   741.0  
[EVENT]검색결과                                                                                                                     1829.5